In [1]:
import numpy
from flask import Flask
from flask import request
from keras.layers import Dense,Dropout,Flatten,MaxPooling2D,Convolution2D,Activation
from keras.models import Sequential
from tensorflow.python.keras.backend import set_session
from keras.models import load_model
from keras.wrappers.scikit_learn import KerasClassifier
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow 
from io import BytesIO
import json
import requests
import cv2
from os  import listdir
app = Flask(__name__)

global graph
graph = tensorflow.get_default_graph()
sess = tensorflow.Session()

#for build evaluvate model
def larger_model_evaluvate():
        model = Sequential()
        model.add(Dense(64,input_dim=2,activation='relu'))
        model.add(Dense(256,activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(128,activation='relu'))
        model.add(Dense(1,activation='sigmoid'))
        model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model
    

def larger_model_crop_image_model():
        model = Sequential()
        model.add(Convolution2D(32,(3,3),input_shape=(64,64,3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Convolution2D(32,(3,3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Convolution2D(16,(3,3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Convolution2D(8,(3,3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Flatten())
        model.add(Dense(64))
        model.add(Activation('relu'))
        model.add(Dropout(0.5))
        model.add(Dense(1))
        model.add(Activation('sigmoid'))
        model.compile(optimizer = 'rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
        return model
     
def larger_model_normal():
        model = Sequential()
        model.add(Convolution2D(32,(3,3),input_shape=(64,64,3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Convolution2D(32,(3,3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Convolution2D(64,(3,3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Flatten())
        model.add(Dense(64))
        model.add(Activation('relu'))
        model.add(Dropout(0.5))
        model.add(Dense(1))
        model.add(Activation('sigmoid'))
        model.compile(optimizer = 'rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
        return model
    
    

#load models
classifire_evaluvate = larger_model_evaluvate()  
classifire_normal = larger_model_normal()  
classifire_crop_image = larger_model_crop_image_model()  
set_session(sess)
classifire_evaluvate.load_weights('evaluate_90.h5') 
classifire_normal.load_weights('93_model.h5')
classifire_crop_image.load_weights('93_preprocess_model.h5')

@app.route("/predict",methods =['POST'])
def predictions(): 
    REQ = request.files["image"].save('request.jpeg')
    #for API 01 ,prediction
    image = tensorflow.keras.preprocessing.image.load_img('request.jpeg',target_size=(64,64))
    input_arr = tensorflow.keras.preprocessing.image.img_to_array(image)
    input_arr = numpy.array([input_arr]) 
    #for API 02, prediction
    img = cv2.imread('request.jpeg')
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU) 
    kernel = numpy.ones((9,9), numpy.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    result = img.copy()
    result = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
    result[:, :, 3] = mask
    cv2.imwrite('request.png', result)
    image_crop = tensorflow.keras.preprocessing.image.load_img('request.png',target_size=(64,64))
    input_arr_crop = tensorflow.keras.preprocessing.image.img_to_array(image_crop)
    input_arr_crop = numpy.array([input_arr_crop])
    
    with graph.as_default():
                 set_session(sess)
                 api_1 =classifire_normal.predict_proba(input_arr)[0][0]
                 api_2 =classifire_crop_image.predict_proba(input_arr_crop)[0][0]                    
                 prediction = classifire_evaluvate.predict([[[api_1,api_2]]])[0][0]          
    return str(prediction*100)

if __name__ == '__main__':
    app.run(debug=False, port=5003)

Using TensorFlow backend.
C:\ProgramData\Anaconda3\envs\deepkeras\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\envs\deepkeras\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\envs\deepkeras\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\envs\deepk

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5003/ (Press CTRL+C to quit)


127.0.0.1 - - [18/Dec/2020 09:14:12] "POST /predict HTTP/1.1" 200 -
